In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
path = '/Users/sabygalvan/Desktop/FinalDL/'

In [ ]:
dataTraining = pd.read_csv(os.path.join(path, 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'dataTesting.csv'), encoding='UTF-8', index_col=0)

In [ ]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [ ]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [ ]:
vect = CountVectorizer(ngram_range=(1, 2), max_features=1000)
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 1000)

In [ ]:
# Definicion de funcion para preprocesar el texto
def preProssText(df, commonWord, numberCommonWords, rareWord, numberRareWords, spellCorrect):
  # Remove punctutation
  df['plot'] = df['plot'].str.replace('[^\w\s]','')
  
  # Lower case
  df['plot'] = df['plot'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  
  # Remove Common words
  if commonWord == True:
    freq = pd.Series(' '.join(df['plot']).split()).value_counts()[:numberCommonWords]
    df['plot'] = df['plot'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
  
  # Remove Rare words
  if rareWord == True:
    freq = pd.Series(' '.join(df['plot']).split()).value_counts()[-numberRareWords:]
    df['plot'] = df['plot'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
  
  # Spelling correction -> Tener cuidado puede cambiar los valores. Ej: "ur" por "or" en vez de "your"
  if spellCorrect == True:
    from textblob import TextBlob
    df['plot'] = df['plot'].apply(lambda x: str(TextBlob(x).correct()))
    
  return df

In [ ]:
# Pre-procesamiento de texto
dataTraining = preProssText(dataTraining, commonWord=True, numberCommonWords=10, rareWord=True, numberRareWords=10, spellCorrect=False)
dataTesting = preProssText(dataTesting, commonWord=True, numberCommonWords=10, rareWord=True, numberRareWords=10, spellCorrect=False)

In [ ]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most story single father who takes eight year ...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,serial killer decides teach secrets satisfying...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,sweden female blackmailer with disfiguring fac...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,friday afternoon new york president tredway co...,['Drama'],7.4
2582,1990,Narrow Margin,los angeles editor publishing house carol hunn...,"['Action', 'Crime', 'Thriller']",6.6


In [ ]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,who meets by fate shall be sealed by fate ther...
4,1978,Midnight Express,true story billy hayes an american college stu...
5,1996,Primal Fear,martin vail left chicago da office become succ...
6,1950,Crisis,husband wife americans dr eugene mrs helen fer...
7,1959,The Tingler,coroner scientist dr warren chapin researching...


In [ ]:
# define a function that accepts text and returns a list of lemmas
import nltk
nltk.download('wordnet')

# define a function that accepts text and returns a list of stem
def split_into_stem(text):
    stemmer = SnowballStemmer('english')
    text = text.lower()
    words = text.split()
    return [stemmer.stem(word) for word in words]

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sabygalvan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def split_into_lemmas(text):
    from nltk.stem import WordNetLemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

In [ ]:
# Text processing
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

In [ ]:
vect = CountVectorizer(analyzer=split_into_lemmas, ngram_range=(1, 2), max_features=1000)
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 1000)

In [ ]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most story single father who takes eight year ...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,serial killer decides teach secrets satisfying...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,sweden female blackmailer with disfiguring fac...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,friday afternoon new york president tredway co...,['Drama'],7.4
2582,1990,Narrow Margin,los angeles editor publishing house carol hunn...,"['Action', 'Crime', 'Thriller']",6.6


In [ ]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [ ]:
y_genres.shape

(7895, 24)

In [ ]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.1, random_state=42)

In [ ]:
# Keras NN
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, Embedding, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import History
from keras.models import Model
from keras.preprocessing import sequence
from keras.layers import Conv1D, GlobalMaxPooling1D

In [ ]:
from keras.preprocessing import sequence
from keras.layers import Conv1D, GlobalMaxPooling1D

In [ ]:
# set parameters:
batch_size = 32
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

# Modelo alternativo
top_words = 1000
max_review_length = 1000

# Set the model
def textModel(optimizer="RMSprop"):
    model = Sequential()
    model.add(Embedding(top_words, 50, input_length=max_review_length))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='sigmoid',
                 strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('sigmoid'))
    model.add(Dense(24, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [ ]:
ix_train=np.random.choice(X_train.shape[0], 1000)
ix_test=np.random.choice(X_test.shape[0], 100)
model=textModel()
model.fit(X_train[ix_train], y_train_genres[ix_train], epochs=2, batch_size=50, validation_data=(X_test[ix_test], y_test_genres[ix_test]))

Train on 1000 samples, validate on 100 samples
Epoch 1/2
1000/1000 [==============================] - 40s 40ms/step - loss: 7.6078 - acc: 0.2330 - val_loss: 7.5742 - val_acc: 0.2700
Epoch 2/2
1000/1000 [==============================] - 34s 34ms/step - loss: 7.3756 - acc: 0.2250 - val_loss: 7.5595 - val_acc: 0.2700


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# Definicion del modelo KerasClassifier
model = KerasClassifier(build_fn=textModel, verbose=0)

# Lista de parametros a evaluar
parametros = {
    "optimizer": ["SGD", "RMSprop", "Adagrad"], 
    "drop": [0.2,0.5,1],
    "neurons": [50,100,256]
    #"kernel": ["random_normal", "he_uniform", "lecun_normal", "he_normal", "glorot_uniform", "glorot_normal", "lecun_uniform", "zeros", "ones"], 
    #
    #"moreHiddenLayers": [True, False]
}

# Busqueda aleatoria entre los parametros
random_search = RandomizedSearchCV(estimator=model, param_distributions=parametros)

In [ ]:
random_result=random_search.fit(X_train, y_train_genres)

KeyboardInterrupt: 

In [ ]:
random_result

In [ ]:
print("Mejor: %f using %s" % (random_result.best_score_, random_result.best_params_))

In [ ]:
from keras.layers import GlobalAveragePooling1D

In [ ]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_text_RF.csv', index_label='ID')

In [ ]:
# Modelo alternativo
top_words = 1000
max_review_length = 1000

# Set the model
def textModel(optimizer="RMSprop"):
    model = Sequential()
    model.add(Embedding(top_words, 50, input_length=max_review_length))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(24, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [ ]:
ix_train=np.random.choice(X_train.shape[0], 1000)
ix_test=np.random.choice(X_test.shape[0], 100)
model=textModel()
model.fit(X_train[ix_train], y_train_genres[ix_train], epochs=2, batch_size=50, validation_data=(X_test[ix_test], y_test_genres[ix_test]))

Train on 1000 samples, validate on 100 samples
Epoch 1/2
1000/1000 [==============================] - 108s 108ms/step - loss: 7.7952 - acc: 0.1960 - val_loss: 7.2161 - val_acc: 0.2300
Epoch 2/2
1000/1000 [==============================] - 98s 98ms/step - loss: 7.5014 - acc: 0.1950 - val_loss: 7.1731 - val_acc: 0.2300
